# 6章：モジュール

## 6.1:モジュールのインポート

In [2]:
import Data.List

numUniques :: (Eq a) => [a] -> Int
numUniques = length . nub

関数の検索にはHoogle(https://www.haskell.org/hoogle/)　を使うといいよ！便利だね！

GHCiからモジュールにアクセスしたい場合は以下のように打てばいいよ．<br>
ghci> :m + Data.List<br>
複数の場合は列挙すればいいよ<br>
ghci> :m + Data.List Data.Nap Data.Set<br>

モジュールの中の特定の関数を選んでインポートもできるよ<br>

In [10]:
import Data.List (nub, sort)

名前の競合を避けるためにどうするか？<br>
①モジュールの中の特定の関数を除外してインポートもできる<br>

In [11]:
import Data.List hiding (nub)

②修飾付きインポート<br>
普通にインポートすると，　競合してエラーでる場合がある．

In [17]:
import Data.Map
filter (< "a") (fromList [(5,"a"), (3,"b")]) == empty

Ambiguous occurrence `filter'
It could refer to either `Data.Map.filter', imported from `Data.Map' (and originally defined in `containers-0.5.6.2:Data.Map.Base')
                      or `IHaskellPrelude.filter', imported from `Prelude' (and originally defined in `GHC.List')

修飾付きインポートを使おう

In [18]:
import qualified Data.Map 
Data.Map.filter (< "a") (fromList [(5,"a"), (3,"b")]) == empty

True

こんな感じで別名もつけられる

In [20]:
import qualified Data.Map as Map
Map.filter (< "a") (fromList [(5,"a"), (3,"b")]) == empty

True

### "." の問題
関数の参照の”．”と，関数合成演算子の"."をどうやって見分けているのだろうか？<br>
→ qualifiedされたモジュールと関数名の間に空白を開けずに置かれた場合は，インポートされた関数とみなす．そうでなければ関数合成とみなす

## 6.2: 標準モジュールの関数で問題を解く

### 6.2.1 : 単語を数える
単語がたくさん含まれた文字列があって，各単語が何回現れるかを求めたい．<br>
①文字列受け取る(words)→②単語をソート(sort)→③グルーピング(group)→④各グループのサイズを数える(map)

あんまり良い書き方ではないけれど．手続きを追っていくとこうなる

In [33]:
import qualified Data.List as List

list = List.words "hoge hoge fuga hoge fuga fuga hoge"
sorted = List.sort list
grouped = List.group sorted
maped  = List.map (\ws -> (head ws, length ws)) grouped

print list
print sorted
print grouped
print maped

["hoge","hoge","fuga","hoge","fuga","fuga","hoge"]

["fuga","fuga","fuga","hoge","hoge","hoge","hoge"]

[["fuga","fuga","fuga"],["hoge","hoge","hoge","hoge"]]

[("fuga",3),("hoge",4)]

##### 関数合成を用いて書くと以下のように書ける

In [34]:
wordNums :: String -> [(String, Int)]
wordNums = List.map (\ws -> (head ws, length ws)) . List.group . List.sort . List.words

Line 2: Use &&&
Found:
\ ws -> (head ws, length ws)
Why not:
head Control.Arrow.&&& length

In [35]:
wordNums "hoge hoge fuga hoge fuga fuga hoge"

[("fuga",3),("hoge",4)]

良いですね！

### 6.2.2 : リスト検索
2つのリストを受け取り，1つ目のリストが，2つ目のリストのどこに現れているか検索．

In [43]:
import qualified Data.List as List

isIn :: (Eq a) => [a] -> [a] -> Bool
isIn needle haystack = List.any (List.isPrefixOf needle)　(List.tails haystack)

In [45]:
isIn "par" "party"

True

In [46]:
isIn "hoge" "party"

False

なかみ

In [47]:
List.tails "party"

["party","arty","rty","ty","y",""]

In [51]:
List.isPrefixOf ["par"] ["party","arty","rty","ty","y",""]

False

In [54]:
List.any (=='k') "Frank Sobotka"

Line 1: Use elem
Found:
List.any (== 'k')
Why not:
elem 'k'

True

Data.Listの中にも isInfixOf　という名前の関数が同様の動作をします．　そうですか

In [56]:
List.isInfixOf "par" "party"

True

### 6.2.3 : シーザー暗号で暗号化
ある文字列が与えられた時，　各文字をアルファベット上で一定の数シフトする．

In [73]:
import qualified Data.List as List
import qualified Data.Char as Char

encode :: Int -> String -> String
encode offset msg = List.map (Char.chr . (+ offset) . Char.ord) msg

decode :: Int -> String -> String
decode shift msg = encode (negate shift) msg

In [60]:
encode 3 "hey mark"

"kh|#pdun"

In [74]:
decode 3 "kh|#pdun"

"hey mark"

なかみ

In [64]:
Char.ord 'a'

97

In [65]:
Char.chr 97

'a'

In [68]:
List.map Char.ord "abcdefg"

[97,98,99,100,101,102,103]

ナイーブにかくとこんなかんじに

In [79]:
decode' :: Int -> String -> String
decode' shift msg = List.map (Char.chr . (+ negate shift) . Char.ord) msg

In [80]:
decode 3 "kh|#pdun"

"hey mark"

### 6.2.4 : 正格な(遅延評価でない)左畳み込み

Preludeの foldl　は大きなリストの畳み込みに対してスタックオーバーフローを起こすことがある．遅延評価なので

In [83]:
IHaskellPrelude.foldl (+) 0 (replicate 10 1)

10

In [ ]:
IHaskellPrelude.foldl (+) 0 (replicate 10000000000 1)

カーネルが死んだ！　このひとでなし！！

例　:　Haskell はどのように　foldl (+) 0 [1,2,3]を評価しているか？<br>
先延ばしにした計算によるスタックがまず構築され，　空リストに到達したところで計算開始．<br>
そのためリストでかすぎるとスタックオーバーフローするよ

こんなん<br>
foldl (+) 0 [1,2,3] =<br>
foldl (+) (0+1) [2,3] =<br>
foldl (+) ((0+1)+2) [3] =<br>
foldl (+) (((0+1)+2)+3) [] =<br>
((0+1)+2)+3 =<br>
(1+2)+3 = <br>
3+3=<br>
6<br>

一方，　Data.Listにある, foldl' は正格評価なのでこの問題は起こらない

In [4]:
import qualified Data.List as List

List.foldl' (+) 0 (replicate 10000000000 1)

10000000000

(少なくとも)死なない！　つよい！

　例えば，foldl (+) 0 [1,2,3]　は以下のように評価されている<br>

foldl' (+) 0 [1,2,3] = <br>
foldl' (+) 1 [2,3] = <br>
foldl' (+) 3 [3] = <br>
foldl' (+) 6 [] = <br>
6

### 6.2.5 : 各桁の数の合計が40になる最初の自然数は何か？

In [6]:
import qualified Data.Char as Char
import qualified Data.List as List

--数を引数に取って，各桁の数の合計を返す
digitSum :: Int -> Int
digitSum = sum . List.map Char.digitToInt . show

In [8]:
digitSum 101

2

Data.List.find は術後関数とリストを因数に取り，リストの中で条件に合致する最初の要素を返す．<br>
返り血がMaybe という型．条件に当てはまるものがなければ Nothingを返す<br>
0個から1個の値を持てる．失敗する可能性のあることを表現するのに使う．

In [9]:
:t List.find

List.find :: forall a (t :: * -> *). Foldable t => (a -> Bool) -> t a -> Maybe a

In [11]:
List.find (> 4) [3,4,5,6,7]

Just 5

In [17]:
List.find (>10) [3,4,5,6,7]

Nothing

In [15]:
:t Nothing

Nothing :: forall a. Maybe a

In [12]:
:t Just 5

Just 5 :: forall a. Num a => Maybe a

## 6.3 : キーから値へのマッピング

### 6.3.1 : 連想リスト(辞書)
キーと値のペアを順序を気にせずリストにしたもの

In [18]:
phoneBook =   
    [("betty","555-2938")  
    ,("bonnie","452-2928")  
    ,("patsy","493-2928")  
    ,("lucille","205-2928")  
    ,("wendy","939-8282")  
    ,("penny","853-2492")  
    ]  

#### キーの値を検索する関数

ナイーブな実装

In [19]:
findKey :: (Eq k) => k -> [(k,v)] -> v  
findKey key xs = snd . head . filter (\(k,v) -> key == k) $ xs  

In [20]:
findKey "betty" phoneBook

"555-2938"

例外でクラッシュ(空リストのheadを取ろうとする)

In [21]:
findKey "hogehoge" phoneBook

Prelude.head: empty list

Maybe型を使って例外処理を書く．何もなければNothing　を返す<br>
再帰を使って書く場合

In [22]:
findKey :: (Eq k) => k -> [(k,v)] -> Maybe v  
findKey key [] = Nothing  
findKey key ((k,v):xs) = if key == k  
                            then Just v  
                            else findKey key xs  

再帰の読み方わすれた（無能

In [23]:
findKey "betty" phoneBook

Just "555-2938"

In [24]:
findKey "hogehoge" phoneBook

Nothing

畳み込みとして実装もできる．　こっちのが可読性高いかも

In [29]:
findKey :: (Eq k) => k -> [(k,v)] -> Maybe v  
findKey key = foldr (\(k,v) acc -> if key == k then Just v else acc) Nothing  

In [30]:
findKey "betty" phoneBook

Just "555-2938"

In [31]:
findKey "hogehoge" phoneBook

Nothing

### 6.3.2 : Data.Map による，連想リストの操作

In [38]:
import qualified Data.Map as Map  

phoneBook :: Map.Map String String
phoneBook = Map.fromList $
    [("betty","555-2938")  
    ,("bonnie","452-2928")  
    ,("patsy","493-2928")  
    ,("lucille","205-2928")  
    ,("wendy","939-8282")  
    ,("penny","853-2492")  
    ]  

Line 2: Redundant $
Found:
Map.fromList $
  [("betty", "555-2938"), ("bonnie", "452-2928"),
   ("patsy", "493-2928"), ("lucille", "205-2928"),
   ("wendy", "939-8282"), ("penny", "853-2492")]
Why not:
Map.fromList
  [("betty", "555-2938"), ("bonnie", "452-2928"),
   ("patsy", "493-2928"), ("lucille", "205-2928"),
   ("wendy", "939-8282"), ("penny", "853-2492")]

Data.Map.fromList による，　　連想リストのData.Mapへの変換

In [34]:
Map.fromList [("betty","555-2938"),("bonnie","452-2928"),("lucille","205-2928")]  

fromList [("betty","555-2938"),("bonnie","452-2928"),("lucille","205-2928")]

重複したkeyがあった場合，後ろのほうの要素が使われる

In [36]:
Map.fromList [("betty","555-2938"),("jon","452-2928"),("betty","205-2928")]  

fromList [("betty","205-2928"),("jon","452-2928")]

Map.fromList の型<br>
型kとvのペアのリストを受け取り，型kをキー，型vを値とするMap(辞書)を返す

In [40]:
:t Map.fromList 

Map.fromList :: forall k a. Ord k => [(k, a)] -> Map k a

Data.Mapによる，　連想リストの操作

サイズを調べる

In [44]:
:t Map.size

Map.size :: forall k a. Map k a -> Int

In [43]:
Map.size phoneBook

6

ほかにもいろいろあるよ！<br>
めんどくさいから省略するよ！！<br>
Haskellが提供するmoduleはとってもクールだね！！

## 6.4 : モジュールの自作

モジュールからは関数をエクスポートする．　<br>モジュールの外で使えるのはモジュールがエクスポートした関数だけ．

### 6.4.1 : 幾何学モジュール
幾何学オブジェクト(sphere, cube, cuboid)の体積と面積を計算するモジュール<br>
Geometry1.hsというファイルを作ります

In [1]:
import Geometry

attempting to use module `Geometry' (./Geometry.hs) which is not loaded

あれ，importできないぞ

内容はこんな感じ<br>
モジュールがエクスポートする関数を指定して，それから関数を追加．<br>

In [1]:
module Geometry
( sphereVolume  
, sphereArea  
, cubeVolume  
, cubeArea  
, cuboidArea  
, cuboidVolume  
) where  
  
sphereVolume :: Float -> Float  
sphereVolume radius = (4.0 / 3.0) * pi * (radius ^ 3)  
  
sphereArea :: Float -> Float  
sphereArea radius = 4 * pi * (radius ^ 2)  
  
cubeVolume :: Float -> Float  
cubeVolume side = cuboidVolume side side side  
  
cubeArea :: Float -> Float  
cubeArea side = cuboidArea side side side  
  
cuboidVolume :: Float -> Float -> Float -> Float  
cuboidVolume a b c = rectangleArea a b * c  
  
cuboidArea :: Float -> Float -> Float -> Float  
cuboidArea a b c = rectangleArea a b * 2 + rectangleArea a c * 2 + rectangleArea c b * 2  
  
rectangleArea :: Float -> Float -> Float  
rectangleArea a b = a * b  

### 6.4.2 : 階層的モジュール
モジュールには階層構造を与えることができる<br>
例として，Geometryを分割して，立体の種類ごとの3つのサブモジュールを持つモジュールに組み替える<br>

Geometry というフォルダの中に，Sphere.hs,Cuboid.hs,Cube.hsを作る．

In [1]:
import Geometry.Sphere

attempting to use module `Geometry.Sphere' (./Geometry/Sphere.hs) which is not loaded

これもダメだ

中身こんな感じ

In [2]:
module Geometry.Sphere  
( volume  
, area  
) where  
  
volume :: Float -> Float  
volume radius = (4.0 / 3.0) * pi * (radius ^ 3)  
  
area :: Float -> Float  
area radius = 4 * pi * (radius ^ 2)  

In [3]:
module Geometry.Cuboid  
( volume  
, area  
) where  
  
volume :: Float -> Float -> Float -> Float  
volume a b c = rectangleArea a b * c  
  
area :: Float -> Float -> Float -> Float  
area a b c = rectangleArea a b * 2 + rectangleArea a c * 2 + rectangleArea c b * 2  
  
rectangleArea :: Float -> Float -> Float  
rectangleArea a b = a * b  

In [4]:
module Geometry.Cube  
( volume  
, area  
) where  
  
import qualified Geometry.Cuboid as Cuboid  
  
volume :: Float -> Float  
volume side = Cuboid.volume side side side  
  
area :: Float -> Float  
area side = Cuboid.area side side side  

In [5]:
import qualified Geometry.Sphere as Sphere  
import qualified Geometry.Cuboid as Cuboid  
import qualified Geometry.Cube as Cube  

In [7]:
Sphere.area 5

314.15927

In [8]:
Sphere.volume 5

523.5988